# 如何在ChatModels中跟踪令牌使用情况

:::info 前提条件

本指南假设您熟悉以下概念：
- [聊天模型](/docs/concepts/chat_models)

:::

跟踪[令牌](/docs/concepts/tokens/)使用情况以计算成本是将应用程序投入生产的重要部分。本指南介绍如何从LangChain模型调用中获取此信息。

本指南需要 `langchain-anthropic` 和 `langchain-openai >= 0.3.11`。

In [ ]:
%pip install -qU langchain-anthropic langchain-openai

:::note 关于OpenAI流式传输的说明

OpenAI的聊天补全API默认情况下不会流式传输令牌使用统计信息（参见API参考文档
[此处](https://platform.openai.com/docs/api-reference/completions/create#completions-create-stream_options))。
要在使用`ChatOpenAI`或`AzureChatOpenAI`进行流式传输时恢复令牌计数，请设置`stream_usage=True`，
如本指南所示。

:::

## 使用LangSmith

您可以使用[LangSmith](https://www.langchain.com/langsmith)帮助跟踪LLM应用程序中的令牌使用情况。请参阅[LangSmith快速入门指南](https://docs.smith.langchain.com/)。

## 使用AIMessage.usage_metadata

许多模型提供商会在聊天生成响应中返回令牌使用信息。在可用时，这些信息将包含在相应模型产生的`AIMessage`对象中。

LangChain的`AIMessage`对象包含[usage_metadata](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessage.html#langchain_core.messages.ai.AIMessage.usage_metadata)属性。当填充时，该属性将是一个包含标准键（例如，`"input_tokens"`和`"output_tokens"`）的[UsageMetadata](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.UsageMetadata.html)字典。它们还将包括关于缓存的令牌使用情况和来自多模态数据的令牌信息。

例子：

**OpenAI**:

In [1]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="gpt-4o-mini")
openai_response = llm.invoke("hello")
openai_response.usage_metadata

{'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17}

**Anthropic**:

In [2]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-haiku-20240307")
anthropic_response = llm.invoke("hello")
anthropic_response.usage_metadata

{'input_tokens': 8, 'output_tokens': 12, 'total_tokens': 20}

### 流式传输

一些提供商在流式传输上下文中支持令牌计数元数据。

#### OpenAI

例如，OpenAI将在流的末尾返回一个带有令牌使用信息的消息[块](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessageChunk.html)。这种行为由`langchain-openai >= 0.1.9`支持，并可通过设置`stream_usage=True`启用。此属性也可以在实例化`ChatOpenAI`时设置。

:::note
默认情况下，流中的最后一个消息块将在消息的`response_metadata`属性中包含一个`"finish_reason"`。如果我们在流式模式中包含令牌使用情况，将在流的末尾添加一个包含使用元数据的额外块，这样`"finish_reason"`会出现在倒数第二个消息块上。
:::


In [4]:
llm = init_chat_model(model="gpt-4o-mini")

aggregate = None
for chunk in llm.stream("hello", stream_usage=True):
    print(chunk)
    aggregate = chunk if aggregate is None else aggregate + chunk

content='' id='run-adb20c31-60c7-43a2-99b2-d4a53ca5f623'
content='Hello' id='run-adb20c31-60c7-43a2-99b2-d4a53ca5f623'
content='!' id='run-adb20c31-60c7-43a2-99b2-d4a53ca5f623'
content=' How' id='run-adb20c31-60c7-43a2-99b2-d4a53ca5f623'
content=' can' id='run-adb20c31-60c7-43a2-99b2-d4a53ca5f623'
content=' I' id='run-adb20c31-60c7-43a2-99b2-d4a53ca5f623'
content=' assist' id='run-adb20c31-60c7-43a2-99b2-d4a53ca5f623'
content=' you' id='run-adb20c31-60c7-43a2-99b2-d4a53ca5f623'
content=' today' id='run-adb20c31-60c7-43a2-99b2-d4a53ca5f623'
content='?' id='run-adb20c31-60c7-43a2-99b2-d4a53ca5f623'
content='' response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini'} id='run-adb20c31-60c7-43a2-99b2-d4a53ca5f623'
content='' id='run-adb20c31-60c7-43a2-99b2-d4a53ca5f623' usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17}


请注意，使用元数据将包含在各个消息块的总和中：

In [5]:
print(aggregate.content)
print(aggregate.usage_metadata)

Hello! How can I assist you today?
{'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17}


要禁用OpenAI的流式令牌计数，请将`stream_usage`设置为False，或从参数中省略它：

In [6]:
aggregate = None
for chunk in llm.stream("hello"):
    print(chunk)

content='' id='run-8e758550-94b0-4cca-a298-57482793c25d'
content='Hello' id='run-8e758550-94b0-4cca-a298-57482793c25d'
content='!' id='run-8e758550-94b0-4cca-a298-57482793c25d'
content=' How' id='run-8e758550-94b0-4cca-a298-57482793c25d'
content=' can' id='run-8e758550-94b0-4cca-a298-57482793c25d'
content=' I' id='run-8e758550-94b0-4cca-a298-57482793c25d'
content=' assist' id='run-8e758550-94b0-4cca-a298-57482793c25d'
content=' you' id='run-8e758550-94b0-4cca-a298-57482793c25d'
content=' today' id='run-8e758550-94b0-4cca-a298-57482793c25d'
content='?' id='run-8e758550-94b0-4cca-a298-57482793c25d'
content='' response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini'} id='run-8e758550-94b0-4cca-a298-57482793c25d'


您还可以通过在实例化聊天模型时设置`stream_usage`来启用流式令牌使用。这在将聊天模型整合到LangChain[链](/docs/concepts/lcel)中时非常有用：在[流式传输中间步骤](/docs/how_to/streaming#using-stream-events)或使用如[LangSmith](https://docs.smith.langchain.com/)这样的跟踪软件时可以监控使用元数据。

请参阅下面的示例，我们将输出结构化为所需的模式，但仍然可以观察从中间步骤流式传输的令牌使用情况。

In [8]:
from pydantic import BaseModel, Field


class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


llm = init_chat_model(
    model="gpt-4o-mini",
    stream_usage=True,
)
# Under the hood, .with_structured_output binds tools to the
# chat model and appends a parser.
structured_llm = llm.with_structured_output(Joke)

async for event in structured_llm.astream_events("Tell me a joke"):
    if event["event"] == "on_chat_model_end":
        print(f'Token usage: {event["data"]["output"].usage_metadata}\n')
    elif event["event"] == "on_chain_end" and event["name"] == "RunnableSequence":
        print(event["data"]["output"])
    else:
        pass

Token usage: {'input_tokens': 79, 'output_tokens': 23, 'total_tokens': 102}

setup='Why was the math book sad?' punchline='Because it had too many problems.'


在对应的[LangSmith跟踪](https://smith.langchain.com/public/fe6513d5-7212-4045-82e0-fefa28bc7656/r)中，令牌使用情况也可以在聊天模型的负载中看到。

## 使用回调函数

:::info 需要 ``langchain-core>=0.3.49``

:::

LangChain实现了一个回调处理程序和上下文管理器，可以跟踪任何返回`usage_metadata`的聊天模型的令牌使用情况。

还有一些特定API的回调上下文管理器，它们为不同的模型维护价格信息，允许实时估算成本。目前这些只为OpenAI API和Bedrock Anthropic API实现，并可在`langchain-community`中使用：

- [get_openai_callback](https://python.langchain.com/api_reference/community/callbacks/langchain_community.callbacks.manager.get_openai_callback.html)
- [get_bedrock_anthropic_callback](https://python.langchain.com/api_reference/community/callbacks/langchain_community.callbacks.manager.get_bedrock_anthropic_callback.html)

下面，我们演示通用的使用元数据回调管理器。我们可以通过配置或作为上下文管理器来跟踪令牌使用情况。

### 通过配置跟踪令牌使用情况

要通过配置跟踪令牌使用情况，实例化一个`UsageMetadataCallbackHandler`并将其传递给配置：

In [17]:
from langchain.chat_models import init_chat_model
from langchain_core.callbacks import UsageMetadataCallbackHandler

llm_1 = init_chat_model(model="openai:gpt-4o-mini")
llm_2 = init_chat_model(model="anthropic:claude-3-5-haiku-latest")

callback = UsageMetadataCallbackHandler()
result_1 = llm_1.invoke("Hello", config={"callbacks": [callback]})
result_2 = llm_2.invoke("Hello", config={"callbacks": [callback]})
callback.usage_metadata

{'gpt-4o-mini-2024-07-18': {'input_tokens': 8,
  'output_tokens': 10,
  'total_tokens': 18,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 0}},
 'claude-3-5-haiku-20241022': {'input_tokens': 8,
  'output_tokens': 21,
  'total_tokens': 29,
  'input_token_details': {'cache_read': 0, 'cache_creation': 0}}}

### 使用上下文管理器跟踪令牌使用情况

您还可以使用`get_usage_metadata_callback`创建一个上下文管理器并在其中聚合使用元数据：

In [19]:
from langchain.chat_models import init_chat_model
from langchain_core.callbacks import get_usage_metadata_callback

llm_1 = init_chat_model(model="openai:gpt-4o-mini")
llm_2 = init_chat_model(model="anthropic:claude-3-5-haiku-latest")

with get_usage_metadata_callback() as cb:
    llm_1.invoke("Hello")
    llm_2.invoke("Hello")
    print(cb.usage_metadata)

{'gpt-4o-mini-2024-07-18': {'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}, 'claude-3-5-haiku-20241022': {'input_tokens': 8, 'output_tokens': 21, 'total_tokens': 29, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}}}


这些方法中的任何一个都将聚合跨多个模型调用的令牌使用情况。例如，您可以在[代理](https://python.langchain.com/docs/concepts/agents/)中使用它来跟踪对一个模型的重复调用的令牌使用情况：

In [ ]:
%pip install -qU langgraph

In [20]:
from langgraph.prebuilt import create_react_agent


# Create a tool
def get_weather(location: str) -> str:
    """Get the weather at a location."""
    return "It's sunny."


callback = UsageMetadataCallbackHandler()

tools = [get_weather]
agent = create_react_agent("openai:gpt-4o-mini", tools)
for step in agent.stream(
    {"messages": [{"role": "user", "content": "What's the weather in Boston?"}]},
    stream_mode="values",
    config={"callbacks": [callback]},
):
    step["messages"][-1].pretty_print()


print(f"\nTotal usage: {callback.usage_metadata}")

================================ Human Message =================================

What's the weather in Boston?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_izMdhUYpp9Vhx7DTNAiybzGa)
 Call ID: call_izMdhUYpp9Vhx7DTNAiybzGa
  Args:
    location: Boston
================================= Tool Message =================================
Name: get_weather

It's sunny.
================================== Ai Message ==================================

The weather in Boston is sunny.

Total usage: {'gpt-4o-mini-2024-07-18': {'input_token_details': {'audio': 0, 'cache_read': 0}, 'input_tokens': 125, 'total_tokens': 149, 'output_tokens': 24, 'output_token_details': {'audio': 0, 'reasoning': 0}}}


## 下一步

您现在已经看到了几个如何跟踪支持的提供商的令牌使用情况的例子。

接下来，查看本节中关于聊天模型的其他操作指南，例如[如何让模型返回结构化输出](/docs/how_to/structured_output)或[如何为聊天模型添加缓存](/docs/how_to/chat_model_caching)。